In [ ]:
from __future__ import absolute_import
from IPython.display import clear_output
import os
from os.path import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from os.path import exists, join

contentPath = os.getcwd()
cv2Path=join(contentPath, 'CV2Images')
genPath=join(contentPath, 'DataGenerator')
testPath=join(contentPath, 'images')
checkpointPath = join(contentPath, 'CheckPoints')
print('ok')

In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=0, patience=1, verbose=1,
    mode='auto', baseline=None, restore_best_weights=True,
    # print('\n',
)

checkpoints = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpointPath,
    monitor='loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)

In [ ]:
IMAGE_SIZE=(224, 224)
BATCH_SIZE = 2

import tensorflow
from tensorflow.keras.utils import image_dataset_from_directory
train_ds = image_dataset_from_directory(
    genPath,
    color_mode='rgb',
    validation_split=0.2,
    subset="training",
    seed=456,
    image_size=(224, 224),
    shuffle=True,
    batch_size=BATCH_SIZE)
print()

validation_ds = image_dataset_from_directory(
    genPath,
    color_mode='rgb',
    validation_split=0.2,
    subset="validation",
    seed=456,
    image_size=(224, 224),
    shuffle=True,
    batch_size=BATCH_SIZE)
print()

test_ds = image_dataset_from_directory(
    testPath,
    color_mode='rgb',
    image_size=(224, 224),
    batch_size = BATCH_SIZE)
# clear_output()

TRAIN_STEPS = 596 // BATCH_SIZE
print('\nTRAIN_STEPS:', TRAIN_STEPS)
VAL_STEPS =  148 // BATCH_SIZE
print('VAL_STEPS:', VAL_STEPS)

In [ ]:
model = tf.keras.Sequential(name='Defcon4_Sequential_V5')
model.add(Conv2D(64, (3,3),
                 activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.build()
model.summary()

In [ ]:
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])
model.built

In [ ]:
history = model.fit(
    x=train_ds,
    y=None,
    batch_size=BATCH_SIZE,
    epochs=5,
    verbose='auto',
    callbacks=[earlyStop, checkpoints],
    validation_split=0.0,
    validation_data=validation_ds,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=TRAIN_STEPS,
    validation_steps=VAL_STEPS,
    validation_batch_size=BATCH_SIZE,
    validation_freq=1,
    max_queue_size=4,
    workers=1,
    use_multiprocessing=True,
)

In [ ]:
modelName = model.name + '.h5'
modelSavePath = join(contentPath, modelName)
model.save(modelSavePath, overwrite=True)

In [ ]:
%ls